In [2]:
import pandas as pd
solar_installs = pd.read_csv('Data/Solar_Footprints_V2_7811899327930675815.csv')

In [3]:
solar_installs.sort_values(by='Shape__Area', ascending=False)

,OBJECTID,County,Acres,Install Type,Urban or Rural,Combined Class,Distance to Substation (Miles) GTET 100 Max Voltage,Percentile (GTET 100 Max Voltage),Substation Name GTET 100 Max Voltage,HIFLD ID (GTET 100 Max Voltage),...,Percentile (GTET 200 Max Voltage),Substation Name GTET 200 Max Voltage,HIFLD ID (GTET 200 Max Voltage),Distance to Substation (Miles) CAISO,Percentile (CAISO),Substation CASIO Name,HIFLD ID (CAISO),Solar Technoeconomic Intersection,Shape__Area,Shape__Length
2771,2772,Riverside County,7438.603255,Ground,Rural,Ground - Rural,3.886829,50th to 75th,Blythe Walc,304454.0,...,50th to 75th,Colorado River,303761.0,6.290021,50th to 75th,Colorado River,303761.0,Outside,3.010313e+07,35928.275147
5387,5388,Kings County,5058.074647,Ground,Rural,Ground - Rural,2.932513,50th to 75th,Unknown,309129.0,...,0 to 25th,Unknown,309129.0,3.605828,25th to 50th,Henrietta,304664.0,Outside,2.046914e+07,21487.606885
2804,2805,Riverside County,3875.067318,Ground,Rural,Ground - Rural,2.513474,25th to 50th,Unknown,300795.0,...,0 to 25th,Unknown,300795.0,4.587985,25th to 50th,Kem,306318.0,Outside,1.568186e+07,22390.045184
3339,3340,San Bernardino County,3444.751330,Ground,Rural,Ground - Rural,30.154816,75th to 100th,Sandy Valley,NaN,...,75th to 100th,Camino,305759.0,0.000000,0 to 25th,Ivanpah,310135.0,Outside,1.394046e+07,23319.291206
1362,1363,Kings County,2894.855803,Ground,Rural,Ground - Rural,1.486667,25th to 50th,Unknown,309129.0,...,0 to 25th,Unknown,309129.0,2.166880,0 to 25th,Henrietta,304664.0,Outside,1.171514e+07,22346.529151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4278,4279,Santa Clara County,0.034271,Rooftop,Urban,Rooftop - Urban,0.876115,0 to 25th,Ames Distribution,310434.0,...,25th to 50th,Ravenswood,308197.0,7.727228,50th to 75th,Scott,300561.0,Within,1.386912e+02,53.207933
5160,5161,Kern County,0.029450,Ground,Rural,Ground - Rural,6.904726,75th to 100th,Whirlwind,306225.0,...,50th to 75th,Whirlwind,306225.0,6.904726,50th to 75th,Whirlwind,306225.0,Outside,1.191793e+02,49.438403
5166,5167,Kern County,0.017946,Ground,Rural,Ground - Rural,6.659845,75th to 100th,Whirlwind,306225.0,...,50th to 75th,Whirlwind,306225.0,6.659845,50th to 75th,Whirlwind,306225.0,Outside,7.262562e+01,34.145689
5167,5168,Kern County,0.007517,Ground,Rural,Ground - Rural,6.266930,75th to 100th,Whirlwind,306225.0,...,50th to 75th,Whirlwind,306225.0,6.266930,50th to 75th,Whirlwind,306225.0,Outside,3.042128e+01,158.366524
